In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [8]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    # dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
    #                                                                 labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    return actions_logits, aloss, dloss, aloss2, (1-dones)

In [9]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [10]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2, self.dones__ = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [11]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [12]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [13]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [14]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [15]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [16]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            idx = np.random.choice(np.arange(memory_size// batch_size))
            states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            states = states[rates == np.max(rates)]
            actions = actions[rates == np.max(rates)]
            next_states = next_states[rates == np.max(rates)]
            rewards = rewards[rates == np.max(rates)]
            dones = dones[rates == np.max(rates)]
            rates = rates[rates == np.max(rates)]
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _ = sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            #print(dones__, np.max(rates), rates.shape, dones)
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:16.0000 R:16.0000 rate:0.0320 aloss:1.3881 dloss:1.0946 aloss2:1.5875 exploreP:0.9984
Episode:1 meanR:20.0000 R:24.0000 rate:0.0480 aloss:1.3802 dloss:1.0829 aloss2:1.6100 exploreP:0.9960
Episode:2 meanR:22.3333 R:27.0000 rate:0.0540 aloss:1.3891 dloss:1.0870 aloss2:1.6304 exploreP:0.9934
Episode:3 meanR:20.2500 R:14.0000 rate:0.0280 aloss:1.3958 dloss:1.0771 aloss2:1.6156 exploreP:0.9920
Episode:4 meanR:19.4000 R:16.0000 rate:0.0320 aloss:1.3806 dloss:1.0640 aloss2:1.6317 exploreP:0.9904
Episode:5 meanR:23.3333 R:43.0000 rate:0.0860 aloss:1.3802 dloss:1.0632 aloss2:1.6433 exploreP:0.9862
Episode:6 meanR:21.7143 R:12.0000 rate:0.0240 aloss:1.4005 dloss:1.0529 aloss2:1.6453 exploreP:0.9851
Episode:7 meanR:21.6250 R:21.0000 rate:0.0420 aloss:1.3754 dloss:1.0507 aloss2:1.6348 exploreP:0.9830
Episode:8 meanR:21.6667 R:22.0000 rate:0.0440 aloss:1.3767 dloss:1.0369 aloss2:1.6379 exploreP:0.9809
Episode:9 meanR:20.7000 R:12.0000 rate:0.0240 aloss:1.3749 dloss:1.0337 aloss2:1.6

Episode:80 meanR:22.7160 R:19.0000 rate:0.0380 aloss:1.3832 dloss:0.6166 aloss2:3.5487 exploreP:0.8336
Episode:81 meanR:22.7561 R:26.0000 rate:0.0520 aloss:1.3807 dloss:0.5908 aloss2:3.7560 exploreP:0.8315
Episode:82 meanR:22.6867 R:17.0000 rate:0.0340 aloss:1.3787 dloss:0.6229 aloss2:3.6886 exploreP:0.8301
Episode:83 meanR:22.5238 R:9.0000 rate:0.0180 aloss:1.3903 dloss:0.5723 aloss2:3.7330 exploreP:0.8293
Episode:84 meanR:22.6588 R:34.0000 rate:0.0680 aloss:1.3745 dloss:0.5879 aloss2:3.8610 exploreP:0.8266
Episode:85 meanR:22.8256 R:37.0000 rate:0.0740 aloss:1.3865 dloss:0.5636 aloss2:3.8491 exploreP:0.8235
Episode:86 meanR:22.6782 R:10.0000 rate:0.0200 aloss:1.3775 dloss:0.6057 aloss2:3.9735 exploreP:0.8227
Episode:87 meanR:22.6136 R:17.0000 rate:0.0340 aloss:1.3777 dloss:0.5790 aloss2:4.0657 exploreP:0.8214
Episode:88 meanR:22.5393 R:16.0000 rate:0.0320 aloss:1.3763 dloss:0.6131 aloss2:4.0535 exploreP:0.8201
Episode:89 meanR:22.4333 R:13.0000 rate:0.0260 aloss:1.3856 dloss:0.6133 a

Episode:159 meanR:33.5600 R:27.0000 rate:0.0540 aloss:1.3683 dloss:0.5487 aloss2:5.2696 exploreP:0.6302
Episode:160 meanR:33.2700 R:66.0000 rate:0.1320 aloss:1.3654 dloss:0.5375 aloss2:5.3290 exploreP:0.6261
Episode:161 meanR:33.4000 R:24.0000 rate:0.0480 aloss:1.3898 dloss:0.5252 aloss2:5.4683 exploreP:0.6246
Episode:162 meanR:34.6700 R:141.0000 rate:0.2820 aloss:1.3668 dloss:0.5269 aloss2:5.3045 exploreP:0.6160
Episode:163 meanR:34.6600 R:18.0000 rate:0.0360 aloss:1.3643 dloss:0.5089 aloss2:5.2325 exploreP:0.6149
Episode:164 meanR:34.8200 R:28.0000 rate:0.0560 aloss:1.3659 dloss:0.5335 aloss2:5.3387 exploreP:0.6132
Episode:165 meanR:35.2200 R:78.0000 rate:0.1560 aloss:1.3708 dloss:0.5059 aloss2:5.3060 exploreP:0.6085
Episode:166 meanR:35.3800 R:34.0000 rate:0.0680 aloss:1.3645 dloss:0.4979 aloss2:5.3597 exploreP:0.6065
Episode:167 meanR:35.5700 R:40.0000 rate:0.0800 aloss:1.3748 dloss:0.5149 aloss2:5.3273 exploreP:0.6041
Episode:168 meanR:35.6500 R:21.0000 rate:0.0420 aloss:1.3669 dl

Episode:238 meanR:41.5300 R:24.0000 rate:0.0480 aloss:1.3760 dloss:0.4944 aloss2:5.3349 exploreP:0.4595
Episode:239 meanR:41.6000 R:25.0000 rate:0.0500 aloss:1.3819 dloss:0.5172 aloss2:5.3114 exploreP:0.4584
Episode:240 meanR:41.5200 R:15.0000 rate:0.0300 aloss:1.3854 dloss:0.4478 aloss2:5.3282 exploreP:0.4577
Episode:241 meanR:41.2800 R:19.0000 rate:0.0380 aloss:1.3807 dloss:0.4644 aloss2:5.3499 exploreP:0.4568
Episode:242 meanR:41.1800 R:19.0000 rate:0.0380 aloss:1.3824 dloss:0.5067 aloss2:5.3832 exploreP:0.4560
Episode:243 meanR:41.0600 R:18.0000 rate:0.0360 aloss:1.3884 dloss:0.5208 aloss2:5.4183 exploreP:0.4552
Episode:244 meanR:41.0100 R:26.0000 rate:0.0520 aloss:1.3783 dloss:0.5244 aloss2:5.3024 exploreP:0.4540
Episode:245 meanR:40.1200 R:24.0000 rate:0.0480 aloss:1.3818 dloss:0.5084 aloss2:5.3564 exploreP:0.4530
Episode:246 meanR:40.1100 R:10.0000 rate:0.0200 aloss:1.3837 dloss:0.5324 aloss2:5.3741 exploreP:0.4525
Episode:247 meanR:38.9500 R:28.0000 rate:0.0560 aloss:1.3826 dlo

Episode:317 meanR:24.1900 R:9.0000 rate:0.0180 aloss:1.3995 dloss:0.5037 aloss2:5.4515 exploreP:0.3952
Episode:318 meanR:24.0800 R:12.0000 rate:0.0240 aloss:1.3929 dloss:0.5078 aloss2:5.3169 exploreP:0.3947
Episode:319 meanR:22.8500 R:10.0000 rate:0.0200 aloss:1.3908 dloss:0.4405 aloss2:5.2660 exploreP:0.3943
Episode:320 meanR:22.4200 R:10.0000 rate:0.0200 aloss:1.3971 dloss:0.4909 aloss2:5.3827 exploreP:0.3939
Episode:321 meanR:22.0700 R:24.0000 rate:0.0480 aloss:1.4004 dloss:0.5045 aloss2:5.3565 exploreP:0.3930
Episode:322 meanR:21.8100 R:16.0000 rate:0.0320 aloss:1.3945 dloss:0.5531 aloss2:5.3465 exploreP:0.3924
Episode:323 meanR:20.6800 R:12.0000 rate:0.0240 aloss:1.3988 dloss:0.5222 aloss2:5.3241 exploreP:0.3920
Episode:324 meanR:20.7300 R:25.0000 rate:0.0500 aloss:1.3950 dloss:0.5505 aloss2:5.3495 exploreP:0.3910
Episode:325 meanR:20.6900 R:10.0000 rate:0.0200 aloss:1.4012 dloss:0.5050 aloss2:5.3340 exploreP:0.3906
Episode:326 meanR:20.5600 R:27.0000 rate:0.0540 aloss:1.3965 dlos

...

Episode:1713 meanR:180.5100 R:10.0000 rate:0.0200 aloss:1.0785 dloss:0.2229 aloss2:42.4114 exploreP:0.0100
Episode:1714 meanR:181.4000 R:405.0000 rate:0.8100 aloss:1.0853 dloss:0.1969 aloss2:39.2388 exploreP:0.0100
Episode:1715 meanR:180.0200 R:362.0000 rate:0.7240 aloss:1.0810 dloss:0.2000 aloss2:37.7217 exploreP:0.0100
Episode:1716 meanR:180.0000 R:10.0000 rate:0.0200 aloss:1.0695 dloss:0.1470 aloss2:37.5092 exploreP:0.0100
Episode:1717 meanR:183.8700 R:418.0000 rate:0.8360 aloss:1.0885 dloss:0.2057 aloss2:37.1186 exploreP:0.0100
Episode:1718 meanR:184.4800 R:500.0000 rate:1.0000 aloss:1.0824 dloss:0.1971 aloss2:37.6104 exploreP:0.0100
Episode:1719 meanR:184.7100 R:34.0000 rate:0.0680 aloss:1.0846 dloss:0.1736 aloss2:39.4570 exploreP:0.0100
Episode:1720 meanR:179.8100 R:10.0000 rate:0.0200 aloss:1.1021 dloss:0.1472 aloss2:33.3640 exploreP:0.0100
Episode:1721 meanR:184.7100 R:500.0000 rate:1.0000 aloss:1.0858 dloss:0.2073 aloss2:38.0007 exploreP:0.0100
Episode:1722 meanR:189.4900 R:50

Episode:1790 meanR:164.6300 R:23.0000 rate:0.0460 aloss:1.1361 dloss:0.2831 aloss2:29.5433 exploreP:0.0100
Episode:1791 meanR:168.8900 R:436.0000 rate:0.8720 aloss:1.1446 dloss:0.2514 aloss2:34.5898 exploreP:0.0100
Episode:1792 meanR:168.8900 R:10.0000 rate:0.0200 aloss:1.1419 dloss:0.1714 aloss2:25.1943 exploreP:0.0100
Episode:1793 meanR:169.0500 R:26.0000 rate:0.0520 aloss:1.1476 dloss:0.2302 aloss2:22.5243 exploreP:0.0100
Episode:1794 meanR:169.2700 R:32.0000 rate:0.0640 aloss:1.1508 dloss:0.2571 aloss2:26.7391 exploreP:0.0100
Episode:1795 meanR:169.5200 R:35.0000 rate:0.0700 aloss:1.1377 dloss:0.2240 aloss2:28.4595 exploreP:0.0100
Episode:1796 meanR:171.1600 R:500.0000 rate:1.0000 aloss:1.1396 dloss:0.2318 aloss2:28.1500 exploreP:0.0100
Episode:1797 meanR:171.2500 R:19.0000 rate:0.0380 aloss:1.1481 dloss:0.2740 aloss2:57.2953 exploreP:0.0100
Episode:1798 meanR:169.8300 R:128.0000 rate:0.2560 aloss:1.1417 dloss:0.2475 aloss2:28.4894 exploreP:0.0100
Episode:1799 meanR:164.9400 R:11.0

Episode:1867 meanR:139.3500 R:500.0000 rate:1.0000 aloss:1.0935 dloss:0.2384 aloss2:40.4735 exploreP:0.0100
Episode:1868 meanR:139.9800 R:81.0000 rate:0.1620 aloss:1.0922 dloss:0.1922 aloss2:41.4139 exploreP:0.0100
Episode:1869 meanR:143.3000 R:343.0000 rate:0.6860 aloss:1.0894 dloss:0.2233 aloss2:42.4991 exploreP:0.0100
Episode:1870 meanR:144.7500 R:163.0000 rate:0.3260 aloss:1.0958 dloss:0.1947 aloss2:41.5232 exploreP:0.0100
Episode:1871 meanR:145.4600 R:81.0000 rate:0.1620 aloss:1.0965 dloss:0.2216 aloss2:43.6253 exploreP:0.0100
Episode:1872 meanR:148.5600 R:328.0000 rate:0.6560 aloss:1.0880 dloss:0.2279 aloss2:43.0366 exploreP:0.0100
Episode:1873 meanR:146.0900 R:111.0000 rate:0.2220 aloss:1.0893 dloss:0.2391 aloss2:45.0120 exploreP:0.0100
Episode:1874 meanR:143.0700 R:137.0000 rate:0.2740 aloss:1.0890 dloss:0.2623 aloss2:43.1629 exploreP:0.0100
Episode:1875 meanR:138.1900 R:12.0000 rate:0.0240 aloss:1.0895 dloss:0.1973 aloss2:50.3468 exploreP:0.0100
Episode:1876 meanR:135.2400 R:1

Episode:1944 meanR:228.9600 R:500.0000 rate:1.0000 aloss:1.1097 dloss:0.2928 aloss2:51.9673 exploreP:0.0100
Episode:1945 meanR:233.5400 R:500.0000 rate:1.0000 aloss:1.1062 dloss:0.2576 aloss2:45.6432 exploreP:0.0100
Episode:1946 meanR:236.4900 R:500.0000 rate:1.0000 aloss:1.1039 dloss:0.2595 aloss2:48.2718 exploreP:0.0100
Episode:1947 meanR:236.2200 R:93.0000 rate:0.1860 aloss:1.1091 dloss:0.2526 aloss2:57.9542 exploreP:0.0100
Episode:1948 meanR:237.1500 R:119.0000 rate:0.2380 aloss:1.1012 dloss:0.2475 aloss2:40.0031 exploreP:0.0100
Episode:1949 meanR:242.0400 R:500.0000 rate:1.0000 aloss:1.1032 dloss:0.2369 aloss2:45.6389 exploreP:0.0100
Episode:1950 meanR:242.8900 R:96.0000 rate:0.1920 aloss:1.1111 dloss:0.2755 aloss2:50.1484 exploreP:0.0100
Episode:1951 meanR:247.2300 R:500.0000 rate:1.0000 aloss:1.1071 dloss:0.2638 aloss2:48.5350 exploreP:0.0100
Episode:1952 meanR:251.2400 R:500.0000 rate:1.0000 aloss:1.1061 dloss:0.2545 aloss2:46.5224 exploreP:0.0100
Episode:1953 meanR:255.3100 R:

Episode:2020 meanR:393.9000 R:221.0000 rate:0.4420 aloss:1.0930 dloss:0.1824 aloss2:58.5152 exploreP:0.0100
Episode:2021 meanR:397.7500 R:500.0000 rate:1.0000 aloss:1.0850 dloss:0.1608 aloss2:56.1892 exploreP:0.0100
Episode:2022 meanR:397.7500 R:500.0000 rate:1.0000 aloss:1.0895 dloss:0.1925 aloss2:56.3656 exploreP:0.0100
Episode:2023 meanR:397.7500 R:500.0000 rate:1.0000 aloss:1.0808 dloss:0.1668 aloss2:51.0290 exploreP:0.0100
Episode:2024 meanR:398.4900 R:500.0000 rate:1.0000 aloss:1.0989 dloss:0.2144 aloss2:59.5213 exploreP:0.0100
Episode:2025 meanR:402.9400 R:500.0000 rate:1.0000 aloss:1.0859 dloss:0.2034 aloss2:49.1122 exploreP:0.0100
Episode:2026 meanR:398.4800 R:54.0000 rate:0.1080 aloss:1.0848 dloss:0.2101 aloss2:47.8940 exploreP:0.0100
Episode:2027 meanR:394.6800 R:120.0000 rate:0.2400 aloss:1.0838 dloss:0.1841 aloss2:53.6323 exploreP:0.0100
Episode:2028 meanR:394.6800 R:500.0000 rate:1.0000 aloss:1.0918 dloss:0.2188 aloss2:54.0296 exploreP:0.0100
Episode:2029 meanR:394.6800 R

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')`

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.